In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-Structuredness") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-Structuredness").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/27 00:48:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/27 00:48:57 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csall.createOrReplaceTempView("CSET")
csall.show(truncate=150)

23/04/27 00:48:59 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                                                                                  pset|       count|
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                     <schema.org/item> <schema.org/position> isa:<schema.org/ListItem>|6.82461002E8|
|                                                                                          <schema.org/itemListElement> isa:<schema.org/BreadcrumbList>|4.56168963E8|
|                                                                   <schema.org/item> <schema.org/name> <schema.org/position> isa:<schema.org/ListItem>| 3.9303227E8|
|   

In [3]:
from pyspark.sql.functions import col
def findISA(topn = None):
    if(topn is not None):
        param = f"limit {topn}"
    else:
        param = ""
    
    sets = csall.withColumn("pset", split(csall["pset"], " "))
    
    distinct_predicate = sets.selectExpr("explode(pset) as predicate", "count") \
                    .groupBy("predicate").agg(f.sum(f.col("count")).alias("count"))  \
                    .createOrReplaceTempView("newsets")
    distinct_predicate=spark.sql(f"select * from newsets where predicate like '%isa:%' and count > 1 order by count desc {param}")    
    return distinct_predicate

In [4]:
distinct_predicate = findISA(10000)
#distinct_predicate.show()
distinct_predicate_list = distinct_predicate.select('predicate').rdd.flatMap(lambda x: x).collect()
print(len(distinct_predicate_list))

10000


In [10]:
def calculate_structuredness(pred, limit):
    # Danger injection SQL
    pred = pred.replace("'", "\\'")
    sets = spark.sql(f"SELECT pset, count FROM CSET WHERE pset LIKE '%{pred}%' limit {limit}")
    sets = sets.withColumn("pset", split(sets["pset"], " "))
    
    if sets.count() <= 1:
        #print(f"{pred}: error")
        return -1
    
    count_sum = sets.agg({"count": "sum"}).collect()[0][0]
    count_used = sets.selectExpr("sum(size(pset) * count) as count_used").collect()[0][0]
    
    distinct_predicate_count = sets.selectExpr("explode(pset) as predicate").distinct().count()

    structuredness = count_used / (count_sum * distinct_predicate_count)
    #print(f"{pred}: {structuredness}")
    return structuredness

In [ ]:
#%%time
from pyspark.sql import Row
rows = []
for pred in distinct_predicate_list:
    rows.append(Row(type=pred, average=float(calculate_structuredness(pred, 10)) ))

df = spark.createDataFrame(rows)

In [18]:
df.write.option("header",True) \
            .mode("overwrite") \
            .csv(f"s3a://test-out/structuredness/limit10top10000")

In [19]:
readtest = spark.read.option("header",True) \
  .csv("s3a://test-out/structuredness/limit10top10000/**")
readtest.show(truncate=150)

+---------------------------------------------------------------------------------------------------------------------------------------+-------+
|                                                                                                                                   type|average|
+---------------------------------------------------------------------------------------------------------------------------------------+-------+
|                   isa:<streetdir.es/ES/Castilla-Y-Leon/Provincia-De-Burgos/Quemada/Strassen/Plaza-Mayor/Restaurantes-Y-Cafes/ListItem>|   -1.0|
|                                    isa:<streetdir.nl/NL/Limburg/Simpelveld/Simpelveld/Strassen/Houbiersstraat/Drankenwinkels/ListItem>|   -1.0|
|                          isa:<sokakrehberi.net/TR/Balikesir/Altieylul/Cities/Koseler/Streets/Balikesir-Kutahya-Yolu/Kitaplar/ListItem>|   -1.0|
|               isa:<streetdir.com/DE/Berlin/Tempelhof-Schoneberg/Schoneberg/Strassen/Bayreuther-Strasse-4597379/Kinos-Theat

In [20]:
readtestsort = readtest.sort(readtest.average.desc())
readtestsort.show(truncate=150)

+--------------------------------------------------+------------------+
|                                              type|           average|
+--------------------------------------------------+------------------+
|                            isa:<schema.orgRating>|0.9999459693105684|
|                      isa:<schema.org/sportsEvent>| 0.999938040083935|
|                       isa:<schema.org/On%20Offer>|0.9998526377836723|
|                  isa:<schema.org/Music%20Website>|0.9998153618906942|
|         isa:<schema.orgOpeningHoursSpecification>|0.9998060512024826|
|                         isa:<schema.org/original>|0.9998018232263178|
|                         isa:<schema.org/Produits>|  0.99970836978711|
|                      isa:<schema.org/FAQPage!!!!>|0.9996220710506425|
|isa:<schema.org/Private%20Joint%20Stock%20Company>|0.9994019138755981|
|                          isa:<schema.org/Deporte>|0.9993638676844784|
|                              isa:<schema.org/Off>|0.9991837686

In [21]:
withouterrors = readtestsort.filter(readtestsort.average != -1)
print(withouterrors.count())
#withouterrors.show(100, truncate=150)

2133
